In [1]:
from ultralytics import YOLO

# Load a model
# model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO('best.pt')  # load a .pt file

# Use the model
# model.train(data="config.yml", epochs=100, amp=False)  # train the model for GPU
# model.train(data="config.yml", epochs=50)  # train the model for CPU
# metrics = model.val()  # evaluate model performance on the validation set


In [2]:
import cv2
import numpy as np
from detect import Coords
from PIL import Image
imgPath = ('./test.jpg')
# imgPath = ('./data/test/images/00047.jpg')

results = model(imgPath)
coords: list[Coords] = list()
for result in results:
    # print(result.boxes)
    for box in result.boxes:
        for xyxy in box.xyxy:
            coord = Coords(xyxy[0], xyxy[1], xyxy[2], xyxy[3])
            coords.append(coord)
    # im_array = result.plot()  # plot a BGR numpy array of predictions
    # im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
    # # im.show()  # show image
    # im.save('results.jpg') 
# print(result["boxes"])
# import cv2
# print(metrics)
# path = model.export(format="onnx") # export the model to ONNX format
# print(path)
# results = model(img)  # predict on an image
# print(results)


image 1/1 c:\Users\jin-yee.chan\Documents\TaxtDetection\test.jpg: 480x640 1 Tag, 72.0ms
Speed: 9.2ms preprocess, 72.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)


In [3]:
for coord in coords:
    coord.drawBoundingBox(imgPath, saveImagePath='draw1.jpg')

In [4]:
for coord in coords:
    # Crop image 
    coord.cropImage(imgPath, saveImagePath='crop.jpg')

In [7]:
from recog import Recog
recog = Recog()
recog.preprocess('crop.jpg', "binary.jpg")
recog.read('binary.jpg')
print(recog.result)


[([[494, 170], [2241, 170], [2241, 302], [494, 302]], 'V T _OE - 2 0 2 01 1 95 5', 0.3846795875551927)]


In [12]:
text = ""
for result in recog.result:
    print(result[1])
    text += result[1] 

V T _OE - 2 0 2 01 1 95 5


In [13]:
text = text.replace(" ", "")
print(text)

VT_OE-202011955


In [16]:
from IPython.display import clear_output, Image, display, HTML
import cv2

# Read one frame from the camera for testing
video = cv2.VideoCapture(0)
_, frame = video.read()
video.release()

import re
from timeit import timeit
import math

extensions=re.findall(r"\\\*(\.\w*)", cv2.imread.__doc__)

def test(extension = ".jpeg"):
    try:
        totalTime=0
        numTry=3
        for _ in range(numTry):
            totalTime+=timeit(lambda: display(Image(data=cv2.imencode(extension, frame)[1])), number=1)
            clear_output(wait=True)
        return totalTime/numTry, extension

    except cv2.error as e: #usually "unsupported file type"
        return (math.inf, extension, e)
for x in sorted(
    [test(extension) for extension in extensions], key=lambda x: x[0]
): print(x)

In [1]:
import cv2
from IPython.display import display, Image

from detect import TrainYOLO

video = cv2.VideoCapture(0)
display_handle=display(None, display_id=True)
model = TrainYOLO("best.pt")
try:
    while True:
        _, frame = video.read()
        frame = cv2.flip(frame, 1) # if your camera reverses your image
        _, frame = cv2.imencode('.jpeg', frame)
        # Detect and draw bounding boxes around tag
        bounding_box: list[Coords] = model.detect(frame, saveImagePath="./draw.jpg")
        for box in bounding_box:
            box.drawBoundingBox("./draw.jpg", "./draw.jpg")
        break
        # Recognize tag and display text
        display_handle.update(Image(data=frame.tobytes()))
except KeyboardInterrupt:
    pass
finally:
    video.release()
    display_handle.update(None)

None

IndexError: tuple index out of range

In [ ]:
cropedImgPath = './crop.jpg'


In [ ]:
import pytesseract
from PIL import Image
print(pytesseract.image_to_string(Image.open(cropedImgPath)))

In [ ]:
import easyocr
reader = easyocr.Reader(['en'])

In [ ]:
result = reader.readtext(x)
print(result)

In [ ]:
# from doctr.models import ocr_predictor
# from doctr.io import DocumentFile
# model = ocr_predictor('db_resnet50', 'crnn_mobilenet_v3_small', pretrained=True)


In [ ]:
# cropedImgPath = './crop.jpg'
# image = DocumentFile.from_images(cropedImgPath)
# out = model(image)
# print(out)

In [ ]:
# import pytesseract
# from PIL import Image
# print(pytesseract.image_to_string(Image.open(cropedImgPath)))